<p style="font-family: times, serif; font-size:16pt; color:blue">Web scraping. Tipología y ciclo de vida de los datos aula 2- PRA1</p>
<div style="font-family: times, serif; font-size:12pt; text-align: right">Baltasar Boix y Yago Ezcurra, abril 2021</div>

In [ ]:
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

<p style="font-family: times, serif; font-size:12pt; color:blue">Descarga de la página Web del Portal oficial de turismo de España:
<a href="https://www.spain.info/es/consulta/monumentos-historicos-espana/">Monumentos de España</a></p>

In [ ]:
def get_list_monumentos(link, params):
    with requests.Session() as s:
        res = requests.get(link, params=params)
        s.cookies.clear()
    if res.status_code==200:
        print('success', res.url)
    else:
        print('fail', res.url)
    #print(res.cookies)
    x = BeautifulSoup(res.text).find_all('div', class_="wrapper-lista-resultados")[0]
    lst1 = [y.get_text() for y in x.find_all('p')]
    lst2 = [y['href'] for y in x.find_all('a')]
    return pd.DataFrame(list(zip(lst1, lst2)), columns =['Name', 'href'])

<p style="font-family: times, serif; font-size:12pt; color:blue">Bucle de lectura de todas las páginas</p>

In [ ]:
df = get_list_monumentos('https://www.spain.info/es/consulta/monumentos-historicos-espana/?reloaded', {})
for i in range(2, 1723//50+1):
    df = df.append(get_list_monumentos('https://www.spain.info/es/consulta/monumentos-historicos-espana/?reloaded', 
                              {'page': i, 'rpp': 50}), ignore_index = True)
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

In [ ]:
df

<p style="font-family: times, serif; font-size:12pt; color:blue">Lectura de datos de cada monumento obtenido en el paso anterior.</p>

In [ ]:
def get_monumento(link):
    link = 'https://www.spain.info' + link
    with requests.Session() as s:
        res = s.get(link)
        s.cookies.clear()
    if res.status_code==200:
        print('success', res.url)
    else:
        print('fail', res.url)
    x0 = BeautifulSoup(res.text)
    x = x0.find_all('script')
    st = re.search(r'(?<=\{)([^}]+)(?=\})', str(x[2])).group(0)
    d = eval('{'+re.sub(' *\n|\t', '', st)+'}')
    if re.search(r'(?<=setView\()([^)]+)(?=\))', str(x[5])) is not None:
        st = re.search(r'(?<=setView\()([^)]+)(?=\))', str(x[5])).group(0)
        d['coordenadas'] = [re.search(r'(?<=\[)([^\]]+)(?=\])', st).group(0).split(',')]
    else:
        d['coordenadas'] = [['ND', 'ND']]
    del d['tipo-recurso']; del d['tipo-plantilla']; del d['zona-turistica']; del d['objetivo']
    del d['url-es']; del d['eje-primario']; del d['eje-secundario']; del d['category']
    if x0.find('section', class_="module entradilla").find('p', class_='text-secundario') != None:
        d['descripcion'] = re.sub(' *\n|\t', '', x0.find('section', 
                                                     class_="module entradilla").
                              find('p', class_='text-secundario').get_text())
    else:
        d['descripcion'] = ''
    x1 = x0.find_all('div', class_="title-box")
    d['direccion'] = [re.sub(' *\n|\t', '', x1[3].p.get_text()).split('\xa0')]
    #print(d)
    return pd.DataFrame.from_dict(d)

In [ ]:
df1 = get_monumento(df.iloc[0,1])
for link in df.iloc[1:,1]:
    df1 = df1.append(get_monumento(link), ignore_index = True)

<p style="font-family: times, serif; font-size:12pt; color:blue">Eliminamos espacios en blanco al principio y final de los campos provincia, comunidad y municipio.</p>

In [ ]:
df1.provincia = df1.provincia.str.strip()
df1.comunidad = df1.comunidad.str.strip()
df1.municipio = df1.municipio.str.strip()
df1.drop('detalle', axis=1, inplace=True)

<p style="font-family: times, serif; font-size:12pt; color:blue">Transformamos las coordenadas en formato texto en los campos latitude y longitude numéricos.</p>

In [ ]:
df1[['latitude', 'longitude']] = pd.DataFrame(df1.coordenadas.to_list(), columns=['latitude', 'longitude'], index=df1.index)
df1.latitude = pd.to_numeric(df1.latitude, errors='coerce')
df1.longitude = pd.to_numeric(df1.longitude, errors='coerce')
df1.drop('coordenadas', axis=1, inplace=True)

<p style="font-family: times, serif; font-size:12pt; color:blue">Fusionamos los dataframes y generamos el fichero <b>monumentos_espana.csv</b>.</p>

In [ ]:
df2 = pd.concat([df,df1], axis=1)

In [ ]:
df2.to_csv('monumentos_espana.csv', index_label='id')